In [3]:
import sys
import os
import argparse
import py3Dmol

def find_list_element_containing_substring(array: list[str, ...], substring):
    for inx, element in enumerate(array):
        if substring in element:
            return inx
    return None


def dispatch_sys_argv():
    CONFIG_FILE = '.config_ipynb'
    if os.path.isfile(CONFIG_FILE):
        with open(CONFIG_FILE, "r") as f:
            sys.argv = f.read().split(" ")
            # if there is a space in the .pdb filename, rebuild sys.argv considering optional argument
            if ".pdb" not in sys.argv[1]:
                extension_idx = find_list_element_containing_substring(sys.argv, ".pdb")
                if extension_idx is not None:
                    path = " ".join(sys.argv[1:extension_idx+1])
                    sys.argv = [sys.argv[0], path, *sys.argv[extension_idx+1:]]
    else:
        sys.exit("error")

    arg_parser = argparse.ArgumentParser()
    arg_parser.add_argument("input_file", help="PDB file input")
    arg_parser.add_argument("--mode", help="choose selection mode")
    arg_parser.add_argument("--cdr3", help="cdr range es: 89-98", required=False)
    args = arg_parser.parse_args()
    pdb_file = args.input_file
    selection_mode = args.mode
    cdr3_range = args.cdr3
    if selection_mode == "CDR" and cdr3_range is None:
        print("CDR mode selected and no cdr3 range is given (with --cdr3). Hardcoded range is going to be used")
    return pdb_file, selection_mode, cdr3_range


def main():
    pdb_file_path, selection_mode, given_cdr3_range = dispatch_sys_argv()

    # load pdb
    with open(pdb_file_path, "r") as ifile:
        raw_pdb_file_content = "".join([x for x in ifile])

    # init view
    view = py3Dmol.view(width=600, height=450)
    view.addModelsAsFrames(raw_pdb_file_content)

    # from alphafoldColab
    if selection_mode == "rainbow":
        view.setStyle({'cartoon': {'color':'spectrum'}})
    elif selection_mode == "CDR":
        if given_cdr3_range is not None:
            cdr3_range = range(*map(int, given_cdr3_range.split("-")))
        else:
            cdr3_range = range(89, 99)
        i = 0
        # from abbas textbook
        cdr1_range = range(26, 38)
        cdr2_range = range(55, 65)

        for line in raw_pdb_file_content.split("\n"):
            split = line.split()
            if len(split) == 0 or split[0] != "ATOM":
                continue
            if int(split[5]) in cdr1_range:
                color = "red"
            elif int(split[5]) in cdr2_range:
                color = "green"
            elif int(split[5]) in cdr3_range:
                color = "blue"
            else:
                color = "gray"
            view.setStyle({'model': -1, 'serial': i+1}, {"cartoon": {'color': color}})
            i += 1
    # view.setStyle({'model': -1}, {"cartoon": {'color': "spectrum"}})
    view.zoomTo()
    view.show()

if __name__ == "__main__":
    main()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol